In [3]:
## Libraries
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PyPDF2 import PdfMerger
import os

In [4]:


class PDFMergerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Encadenador de PDF")
        self.root.geometry("600x400")

        # Lista para almacenar los archivos PDF
        self.pdf_files = []

        # Crear el panel principal
        self.create_widgets()

    def create_widgets(self):
        # Marco principal
        main_frame = tk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Panel de lista de archivos
        list_frame = tk.LabelFrame(main_frame, text="Archivos a encadenar")
        list_frame.pack(fill=tk.BOTH, expand=True)

        # ListBox para mostrar los archivos
        self.listbox = tk.Listbox(list_frame, selectmode=tk.SINGLE)
        self.listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Scrollbar para el ListBox
        scrollbar = tk.Scrollbar(list_frame, orient=tk.VERTICAL)
        scrollbar.config(command=self.listbox.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.listbox.config(yscrollcommand=scrollbar.set)

        # Botones de control
        button_frame = tk.Frame(main_frame)
        button_frame.pack(fill=tk.X, pady=5)

        btn_add = tk.Button(button_frame, text="Agregar", command=self.add_pdf)
        btn_add.pack(side=tk.LEFT, padx=5)

        btn_remove = tk.Button(button_frame, text="Eliminar", command=self.remove_pdf)
        btn_remove.pack(side=tk.LEFT, padx=5)

        btn_up = tk.Button(button_frame, text="Subir", command=self.move_up)
        btn_up.pack(side=tk.LEFT, padx=5)

        btn_down = tk.Button(button_frame, text="Bajar", command=self.move_down)
        btn_down.pack(side=tk.LEFT, padx=5)

        btn_merge = tk.Button(button_frame, text="Encadenar PDFs", command=self.merge_pdfs)
        btn_merge.pack(side=tk.RIGHT, padx=5)

        # Permitir arrastrar y soltar archivos
        self.listbox.drop_target_register(tk.DND_FILES)
        self.listbox.dnd_bind('<<Drop>>', self.on_drop)

    def add_pdf(self):
        """Agrega un archivo PDF a la lista."""
        files = filedialog.askopenfilenames(
            title="Seleccionar archivos PDF",
            filetypes=[("Archivos PDF", "*.pdf")],
        )
        for file in files:
            if file not in self.pdf_files:
                self.pdf_files.append(file)
                self.listbox.insert(tk.END, os.path.basename(file))
            else:
                messagebox.showwarning("Advertencia", f"El archivo '{os.path.basename(file)}' ya está en la lista.")

    def remove_pdf(self):
        """Elimina un archivo PDF seleccionado de la lista."""
        selection = self.listbox.curselection()
        if selection:
            index = selection[0]
            self.listbox.delete(index)
            self.pdf_files.pop(index)

    def move_up(self):
        """Mueve el archivo seleccionado hacia arriba en la lista."""
        selection = self.listbox.curselection()
        if selection and selection[0] > 0:
            index = selection[0]
            self.pdf_files[index], self.pdf_files[index - 1] = self.pdf_files[index - 1], self.pdf_files[index]
            self.update_listbox()

    def move_down(self):
        """Mueve el archivo seleccionado hacia abajo en la lista."""
        selection = self.listbox.curselection()
        if selection and selection[0] < len(self.pdf_files) - 1:
            index = selection[0]
            self.pdf_files[index], self.pdf_files[index + 1] = self.pdf_files[index + 1], self.pdf_files[index]
            self.update_listbox()

    def update_listbox(self):
        """Actualiza el ListBox para reflejar cambios en la lista de archivos."""
        self.listbox.delete(0, tk.END)
        for file in self.pdf_files:
            self.listbox.insert(tk.END, os.path.basename(file))

    def on_drop(self, event):
        """Maneja el evento de arrastrar y soltar archivos."""
        files = self.root.tk.splitlist(event.data)
        for file in files:
            if file.lower().endswith(".pdf"):
                if file not in self.pdf_files:
                    self.pdf_files.append(file)
                    self.listbox.insert(tk.END, os.path.basename(file))
                else:
                    messagebox.showwarning("Advertencia", f"El archivo '{os.path.basename(file)}' ya está en la lista.")
            else:
                messagebox.showwarning("Advertencia", f"El archivo '{os.path.basename(file)}' no es un PDF válido.")

    def merge_pdfs(self):
        """Encadena los archivos PDF en un único archivo."""
        if not self.pdf_files:
            messagebox.showwarning("Advertencia", "No hay archivos para encadenar.")
            return

        output_file = filedialog.asksaveasfilename(
            title="Guardar archivo PDF combinado",
            defaultextension=".pdf",
            filetypes=[("Archivos PDF", "*.pdf")],
        )

        if not output_file:
            return

        merger = PdfMerger()
        try:
            for file in self.pdf_files:
                merger.append(file)
            merger.write(output_file)
            merger.close()
            messagebox.showinfo("Éxito", "El archivo PDF ha sido creado exitosamente.")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo crear el archivo PDF: {str(e)}")

# if __name__ == "__main__":
#     root = tk.Tk()
#     app = PDFMergerApp(root)
#     root.mainloop()

In [5]:
root = tk.Tk()
app = PDFMergerApp(root)
root.mainloop()

AttributeError: 'Listbox' object has no attribute 'drop_target_register'

Puedes hacer un programa en python que encadene uno o más archivos .PDF en un único y nuevo pdf. Que dicho programa tenga una interfaz gráfica y que los pdf a encadenar puedan ser arrastrados a un panel de dicha interfaz grafica. Que los archivos se encadene en el orden que aparecen en dicho panel de arriba hacia abajo. Y que el orden de dichos pdf se pueda mover dentro del panel, o sea que tenga funcionalidad para mover archivos hacia arriba o hacia abajo dentro del  panel. Y además si agrego un archivo que ya está en la lista a encadenar aparezca un mensaje de Warning advirtiendo esa situación, pero que no me impida adicionarlo. Además que permita también eliminar archivos de la lista de panel

A continuación, te proporciono un programa en Python que cumple con los requisitos que has solicitado. Este programa utiliza la biblioteca `tkinter` para la interfaz gráfica y `PyPDF2` para manipular los archivos PDF.

### Código del programa:

```python
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PyPDF2 import PdfMerger
import os

class PDFMergerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Encadenador de PDF")
        self.root.geometry("600x400")

        # Lista para almacenar los archivos PDF
        self.pdf_files = []

        # Crear el panel principal
        self.create_widgets()

    def create_widgets(self):
        # Marco principal
        main_frame = tk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Panel de lista de archivos
        list_frame = tk.LabelFrame(main_frame, text="Archivos a encadenar")
        list_frame.pack(fill=tk.BOTH, expand=True)

        # ListBox para mostrar los archivos
        self.listbox = tk.Listbox(list_frame, selectmode=tk.SINGLE)
        self.listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Scrollbar para el ListBox
        scrollbar = tk.Scrollbar(list_frame, orient=tk.VERTICAL)
        scrollbar.config(command=self.listbox.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.listbox.config(yscrollcommand=scrollbar.set)

        # Botones de control
        button_frame = tk.Frame(main_frame)
        button_frame.pack(fill=tk.X, pady=5)

        btn_add = tk.Button(button_frame, text="Agregar", command=self.add_pdf)
        btn_add.pack(side=tk.LEFT, padx=5)

        btn_remove = tk.Button(button_frame, text="Eliminar", command=self.remove_pdf)
        btn_remove.pack(side=tk.LEFT, padx=5)

        btn_up = tk.Button(button_frame, text="Subir", command=self.move_up)
        btn_up.pack(side=tk.LEFT, padx=5)

        btn_down = tk.Button(button_frame, text="Bajar", command=self.move_down)
        btn_down.pack(side=tk.LEFT, padx=5)

        btn_merge = tk.Button(button_frame, text="Encadenar PDFs", command=self.merge_pdfs)
        btn_merge.pack(side=tk.RIGHT, padx=5)

        # Permitir arrastrar y soltar archivos
        self.listbox.drop_target_register(tk.DND_FILES)
        self.listbox.dnd_bind('<<Drop>>', self.on_drop)

    def add_pdf(self):
        """Agrega un archivo PDF a la lista."""
        files = filedialog.askopenfilenames(
            title="Seleccionar archivos PDF",
            filetypes=[("Archivos PDF", "*.pdf")],
        )
        for file in files:
            if file not in self.pdf_files:
                self.pdf_files.append(file)
                self.listbox.insert(tk.END, os.path.basename(file))
            else:
                messagebox.showwarning("Advertencia", f"El archivo '{os.path.basename(file)}' ya está en la lista.")

    def remove_pdf(self):
        """Elimina un archivo PDF seleccionado de la lista."""
        selection = self.listbox.curselection()
        if selection:
            index = selection[0]
            self.listbox.delete(index)
            self.pdf_files.pop(index)

    def move_up(self):
        """Mueve el archivo seleccionado hacia arriba en la lista."""
        selection = self.listbox.curselection()
        if selection and selection[0] > 0:
            index = selection[0]
            self.pdf_files[index], self.pdf_files[index - 1] = self.pdf_files[index - 1], self.pdf_files[index]
            self.update_listbox()

    def move_down(self):
        """Mueve el archivo seleccionado hacia abajo en la lista."""
        selection = self.listbox.curselection()
        if selection and selection[0] < len(self.pdf_files) - 1:
            index = selection[0]
            self.pdf_files[index], self.pdf_files[index + 1] = self.pdf_files[index + 1], self.pdf_files[index]
            self.update_listbox()

    def update_listbox(self):
        """Actualiza el ListBox para reflejar cambios en la lista de archivos."""
        self.listbox.delete(0, tk.END)
        for file in self.pdf_files:
            self.listbox.insert(tk.END, os.path.basename(file))

    def on_drop(self, event):
        """Maneja el evento de arrastrar y soltar archivos."""
        files = self.root.tk.splitlist(event.data)
        for file in files:
            if file.lower().endswith(".pdf"):
                if file not in self.pdf_files:
                    self.pdf_files.append(file)
                    self.listbox.insert(tk.END, os.path.basename(file))
                else:
                    messagebox.showwarning("Advertencia", f"El archivo '{os.path.basename(file)}' ya está en la lista.")
            else:
                messagebox.showwarning("Advertencia", f"El archivo '{os.path.basename(file)}' no es un PDF válido.")

    def merge_pdfs(self):
        """Encadena los archivos PDF en un único archivo."""
        if not self.pdf_files:
            messagebox.showwarning("Advertencia", "No hay archivos para encadenar.")
            return

        output_file = filedialog.asksaveasfilename(
            title="Guardar archivo PDF combinado",
            defaultextension=".pdf",
            filetypes=[("Archivos PDF", "*.pdf")],
        )

        if not output_file:
            return

        merger = PdfMerger()
        try:
            for file in self.pdf_files:
                merger.append(file)
            merger.write(output_file)
            merger.close()
            messagebox.showinfo("Éxito", "El archivo PDF ha sido creado exitosamente.")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo crear el archivo PDF: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = PDFMergerApp(root)
    root.mainloop()
```

---

### Explicación del código:

1. **Interfaz gráfica**:
   - Se utiliza `tkinter` para crear una interfaz con un `ListBox` que muestra los archivos PDF seleccionados.
   - Los botones permiten agregar, eliminar, mover hacia arriba/abajo y encadenar los archivos.

2. **Arrastrar y soltar**:
   - El `ListBox` admite arrastrar y soltar archivos directamente desde el explorador de archivos.

3. **Advertencia para archivos duplicados**:
   - Si un archivo ya está en la lista, se muestra un mensaje de advertencia, pero no se impide su adición.

4. **Reorganización de archivos**:
   - Los botones "Subir" y "Bajar" permiten cambiar el orden de los archivos en la lista.

5. **Encadenamiento de PDFs**:
   - Se utiliza `PyPDF2.PdfMerger` para combinar los archivos PDF en un único archivo.

6. **Validaciones**:
   - Se verifica que los archivos sean PDF antes de agregarlos.
   - Se manejan errores durante el proceso de combinación.

---

### Requisitos previos:

Para ejecutar este programa, necesitas instalar las siguientes bibliotecas:

```bash
pip install PyPDF2
```

---

### Ejecución:

Guarda el código en un archivo, por ejemplo, `pdf_merger.py`, y ejecútalo:

```bash
python pdf_merger.py
```

Este programa te permitirá gestionar y combinar archivos PDF de manera intuitiva.